#caption with image id csv download

In [ ]:
!pip install googletrans==4.0.0-rc1


In [ ]:
import os
import json
import requests
import pandas as pd
import zipfile
from tqdm import tqdm
from googletrans import Translator
from google.colab import files
from concurrent.futures import ThreadPoolExecutor, as_completed


In [ ]:
# Step 1: Download the MS COCO 2017 annotations zip file
# Use wget to download the annotations
!wget -q http://images.cocodataset.org/annotations/annotations_trainval2017.zip


In [ ]:
# Step 2: Unzip the downloaded file (use -o to overwrite without prompts)
!unzip -o annotations_trainval2017.zip -d annotations

Archive:  annotations_trainval2017.zip
  inflating: annotations/annotations/instances_train2017.json  
  inflating: annotations/annotations/instances_val2017.json  
  inflating: annotations/annotations/captions_train2017.json  
  inflating: annotations/annotations/captions_val2017.json  
  inflating: annotations/annotations/person_keypoints_train2017.json  
  inflating: annotations/annotations/person_keypoints_val2017.json  


In [ ]:
import json
import pandas as pd

# Step 1: Load the COCO captions file
annotations_file = 'annotations/annotations/captions_train2017.json'
with open(annotations_file, 'r') as f:
    coco_data = json.load(f)

# Step 2: Extract image info and captions correctly
images_info = {image['id']: image['file_name'] for image in coco_data['images']}

# Step 3: Create a dictionary to keep only one caption per image (removing duplicates)
captions_info = {}
for caption in coco_data['annotations']:
    image_id = caption['image_id']
    if image_id not in captions_info:  # Only keep the first caption encountered
        captions_info[image_id] = caption['caption']

# Step 4: Create a DataFrame with correct mapping
df_captions = pd.DataFrame(list(captions_info.items()), columns=['image_id', 'caption'])
df_captions['image_file_name'] = df_captions['image_id'].map(images_info)
df_captions['image_url'] = df_captions['image_file_name'].apply(lambda x: f"http://images.cocodataset.org/train2017/{x}")

# Limit to the first 10 images (you can increase this as needed)
df_captions = df_captions.head(20000)

# Step 5: Save the processed captions to a CSV
captions_csv = "mscoco_captions_filtered.csv"
df_captions.to_csv(captions_csv, index=False)

# Step 6: Optional - Download the CSV (this part depends on your environment)
files.download(captions_csv)  # Uncomment this line if you're in a Colab-like environment

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#image download

In [ ]:
import os
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import zipfile

# Step 5: Function to download exact images from the URLs
os.makedirs("mscoco_images", exist_ok=True)

def download_image(url, file_path):
    try:
        response = requests.get(url, stream=True, timeout=5)
        if response.status_code == 200:
            with open(file_path, 'wb') as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)
        else:
            print(f"Failed to download image from {url}")
    except Exception as e:
        print(f"Error downloading {url}: {e}")

def download_images_from_csv(csv_file, num_images=50, num_workers=10):
    df = pd.read_csv(csv_file)

    # Limit the number of images to download to the specified `num_images`
    df = df.head(num_images)

    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        futures = []
        for index, row in df.iterrows():
            image_url = row['image_url']  # Ensure this URL corresponds to the image file name
            image_path = f"mscoco_images/{row['image_file_name']}"
            futures.append(executor.submit(download_image, image_url, image_path))

        for future in tqdm(as_completed(futures), total=len(futures)):
            pass

# Step 6: Download images
number_of_images_to_download =20000  # Adjust this number as needed
download_images_from_csv("mscoco_captions_filtered.csv", num_images=number_of_images_to_download)

# Step 7: Zip the images folder for downloading
zip_filename = "mscoco_images.zip"
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for root, _, filenames in os.walk("mscoco_images"):
        for filename in filenames:
            zipf.write(os.path.join(root, filename))

# Uncomment this if you're in a Jupyter/Colab environment and want to download the zip directly
files.download(zip_filename)


100%|██████████| 20000/20000 [35:16<00:00,  9.45it/s]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>